---
### Check if code is running in google colab or Kaggle

In [ ]:
import os

try:
  import google.colab
  IN_COLAB = True
  os.environ['colab_or_kaggle'] = 'True'
except:
  IN_COLAB = False # Currently only kaggle is supported in exception
  os.environ['colab_or_kaggle'] = 'False'

---
## Download kaggle data in colab

In [ ]:
if IN_COLAB:
  # Info on how to get your api key (kaggle.json) here: https://github.com/Kaggle/kaggle-api#api-credentials
  !pip install --upgrade --force-reinstall --no-deps kaggle
  api_token = {'username':'haldarankit','key':'e4a3e87eae6be921d669fdcbfee2d2bc'}

  !mkdir -p /root/.kaggle/

  import json
  with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)
  !chmod 600 /root/.kaggle/kaggle.json

In [ ]:
%%bash
competition_name='severstal-steel-defect-detection'
project_git_folder='kaggle_severstal'

# if in google colab
if [ $colab_or_kaggle == 'True' ]
then
  # Create folders similar to kaggle execution in colab
  mkdir -p '/content/kaggle'
  cd '/content/kaggle'

  data_path='/content/kaggle/input'
  code_path='/content/kaggle/working'
  echo $data_path
  mkdir -p ${data_path}
  mkdir -p ${code_path}

  # set kaggle data download path
  kaggle config set -n path -v $data_path

  # Download kaggle dataset in kaggle kernel format
  kaggle competitions download -c $competition_name -p $data_path
  unzip -q $data_path/$competition_name.zip -d $data_path/$competition_name/
  rm -rfv $data_path/$competition_name.zip

# if not in google colab (exception - currently using only kaggle)
# change if needed
else
  code_path='/kaggle/working'
fi


# download github repo
cd ${code_path}

MOVE_TO_BRANCH='main'

# Delete in directory exists
if [ -d ${project_git_folder} ]; then
  rm -rf ${project_git_folder}
fi

# clone git repo
git clone https://github.com/ankithaldar/${project_git_folder}.git

# get into kaggle git repo
cd ${project_git_folder}

# more to another branch
git checkout -b $MOVE_TO_BRANCH refs/remotes/origin/$MOVE_TO_BRANCH
git pull

# check its contents
ls -al

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
if IN_COLAB:
  %cd /content/kaggle/working/kaggle_severstal/src
else:
  %cd /kaggle/working/kaggle_severstal/src

---
## 01. Train Models

In [ ]:
!python train.py --params_yml "./config_params/classification_efficientnetb1_fold_01.yml"

---
## 02. Delete code from backend

In [ ]:
if IN_COLAB:
  %rm -rf /content/kaggle/working/kaggle_severstal/*
  %rm -rf /content/kaggle/working/kaggle_severstal
else:
  %rm -rf /kaggle/working/kaggle_severstal/*
  %rm -rf /kaggle/working/kaggle_severstal